# 재현 데이터(Synthetic Data) 생성 with Local LLM

## 목적
- 마스킹된 데이터의 플레이스홀더를 가상 데이터로 대체
- 외부 공유 가능한 자연스러운 재현 데이터 생성

## 변환 대상
| 플레이스홀더 | 변환 예시 |
|-------------|----------|
| [NAME] | 김철수, 이영희 등 |
| [EMAIL] | user001@example.com |
| [CONDITION] | 고혈압, 당뇨 등 |
| [HEALTH_VALUE] | 120mg/dL, 130/85mmHg 등 |
| [DATE] | 2024년 6월 15일 등 |
| [RESULT] | 정상A, 주의 등 |

## 1. 환경 설정

In [2]:
import os
import re
import pandas as pd
import requests
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434")
MODEL_NAME = "gemma3:27b"

print(f"Ollama URL: {OLLAMA_URL}")
print(f"Model: {MODEL_NAME}")

Ollama URL: http://171.0.53.81:11434
Model: gemma3:27b


In [3]:
# Ollama 서버 연결 확인
try:
    response = requests.get(f"{OLLAMA_URL}/api/tags")
    if response.status_code == 200:
        models = response.json().get("models", [])
        model_names = [m["name"] for m in models]
        print("Ollama 서버 연결 성공!")
        print(f"사용 가능한 모델: {model_names}")
        if not any(MODEL_NAME in m for m in model_names):
            print(f"\n[경고] {MODEL_NAME} 모델이 없습니다.")
            print(f"다운로드: ollama pull {MODEL_NAME}")
except requests.exceptions.ConnectionError:
    print("Ollama 서버에 연결할 수 없습니다.")
    print("실행 확인: ollama serve")

Ollama 서버 연결 성공!
사용 가능한 모델: ['qwen2.5:72b', 'alibayram/medgemma:27b', 'qwen3:32b', 'qwen3:30b', 'qwen3:14b', 'qwen3:8b', 'embeddinggemma:300m', 'qwen3-coder:30b', 'deepseek-r1:70b', 'deepseek-r1:32b', 'bge-m3:567m', 'gemma3:4b', 'gemma3:27b', 'gemma3:12b', 'gpt-oss:120b', 'gpt-oss:20b']


## 2. LLM 설정

In [4]:
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

llm = ChatOllama(
    model=MODEL_NAME,
    base_url=OLLAMA_URL,
    temperature=0.7,  # 다양한 가상 데이터 생성을 위해 약간 높게
)

print(f"{MODEL_NAME} 모델 설정 완료!")

gemma3:27b 모델 설정 완료!


## 3. 재현 데이터 생성 프롬프트 및 함수 정의

In [5]:
SYNTHETIC_SYSTEM_PROMPT = """당신은 재현 데이터 생성 전문가입니다.
마스킹된 텍스트의 플레이스홀더를 자연스러운 가상 데이터로 대체하세요.

## 플레이스홀더 변환 규칙

### 1. 직접 식별정보
- [NAME] -> 한국인 이름 (예: 김민수, 이서연, 박지훈)
- [EMAIL] -> 가상 이메일 (예: user123@example.com)
- [PHONE] -> 가상 전화번호 (예: 010-1234-5678)
- [ADDRESS] -> 가상 주소 (예: 서울시 강남구 테헤란로 123)
- [SSN] -> 가상 주민번호 형식 (예: 900101-1******)

### 2. 민감정보 (건강)
- [CONDITION] -> 일반적인 질환명 (예: 고혈압, 당뇨, 고지혈증)
- [HEALTH_VALUE] -> 현실적인 검사 수치 (예: 125mg/dL, 135/88mmHg)
- [RESULT] -> 검진 결과 (예: 정상A, 정상B, 경계, 주의)
- [DATE] -> 가상 날짜 (예: 2024년 5월 10일)
- [MEDICATION] -> 일반적인 약물명 (예: 혈압약, 당뇨약)

## 규칙
1. 문맥에 맞는 자연스러운 데이터를 생성하세요.
2. 같은 텍스트 내에서 [NAME]이 여러 번 나오면 동일한 이름을 사용하세요.
3. 건강 수치는 현실적인 범위 내에서 생성하세요.
4. 플레이스홀더가 아닌 부분은 절대 수정하지 마세요.
5. 변환된 텍스트만 출력하세요. 설명이나 주석은 추가하지 마세요.

## 예시

입력: [NAME]님, [DATE] 건강검진 결과 공복혈당이 [HEALTH_VALUE]로 [CONDITION]이 경계 수준입니다.
출력: 박지민님, 2024년 6월 20일 건강검진 결과 공복혈당이 118mg/dL로 당뇨가 경계 수준입니다.

입력: [EMAIL] 계정으로 로그인하셨습니다.
출력: health_user42@example.com 계정으로 로그인하셨습니다.
"""

In [6]:
def create_synthetic_prompt(text: str) -> str:
    """재현 데이터 생성 프롬프트 생성"""
    return f"""다음 마스킹된 텍스트의 플레이스홀더를 가상 데이터로 대체하세요.

마스킹된 텍스트:
{text}

재현 데이터:"""


def generate_synthetic_text(text: str) -> str:
    """재현 데이터 생성"""
    if not text or pd.isna(text):
        return text
    
    # 플레이스홀더가 없으면 원본 반환
    placeholders = ["[NAME]", "[EMAIL]", "[PHONE]", "[ADDRESS]", "[SSN]",
                   "[CONDITION]", "[HEALTH_VALUE]", "[RESULT]", "[DATE]", "[MEDICATION]"]
    if not any(ph in str(text) for ph in placeholders):
        return text
    
    try:
        messages = [
            SystemMessage(content=SYNTHETIC_SYSTEM_PROMPT),
            HumanMessage(content=create_synthetic_prompt(text))
        ]
        response = llm.invoke(messages)
        return response.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return text

## 4. 재현 데이터 생성 테스트

In [7]:
test_masked = "[NAME]님, [DATE] 건강검진 결과 식전혈당이 [HEALTH_VALUE]로 [CONDITION]가 약양성입니다."

print("=" * 50)
print("[마스킹된 텍스트]")
print(test_masked)
print("=" * 50)
print("[재현 데이터]")
print(generate_synthetic_text(test_masked))
print("=" * 50)

[마스킹된 텍스트]
[NAME]님, [DATE] 건강검진 결과 식전혈당이 [HEALTH_VALUE]로 [CONDITION]가 약양성입니다.
[재현 데이터]
김민수님, 2024년 5월 15일 건강검진 결과 식전혈당이 130mg/dL로 당뇨가 약양성입니다.


In [8]:
# 추가 테스트: 복잡한 케이스
test_complex = """[NAME]님, 안녕하세요. [DATE] 건강검진 결과를 바탕으로 건강 상태를 살펴보겠습니다.

현재 가장 주의가 필요한 부분은 [CONDITION]입니다. 검사 수치가 [HEALTH_VALUE]로 나타났으며,
전반적인 판정은 [RESULT]입니다. 30대 여성의 경우 꾸준한 관리가 중요합니다."""

print("[복잡한 마스킹 텍스트]")
print(test_complex)
print("\n" + "=" * 50 + "\n")
print("[재현 데이터]")
print(generate_synthetic_text(test_complex))

[복잡한 마스킹 텍스트]
[NAME]님, 안녕하세요. [DATE] 건강검진 결과를 바탕으로 건강 상태를 살펴보겠습니다.

현재 가장 주의가 필요한 부분은 [CONDITION]입니다. 검사 수치가 [HEALTH_VALUE]로 나타났으며,
전반적인 판정은 [RESULT]입니다. 30대 여성의 경우 꾸준한 관리가 중요합니다.


[재현 데이터]
김민지님, 안녕하세요. 2024년 7월 15일 건강검진 결과를 바탕으로 건강 상태를 살펴보겠습니다.

현재 가장 주의가 필요한 부분은 고혈압입니다. 검사 수치가 140/90mmHg로 나타났으며,
전반적인 판정은 경계입니다. 30대 여성의 경우 꾸준한 관리가 중요합니다.


## 5. 마스킹된 CSV 데이터 로드

In [ ]:
# 마스킹된 CSV 파일 로드
masked_csv_path = "MESSAGE_INFO_MASKED.csv"  # 마스킹된 파일 경로

df_masked = pd.read_csv(masked_csv_path)

print(f"총 {len(df_masked)}개 메시지")
print(f"세션 수: {df_masked['session_id'].nunique()}개")
df_masked.head()

총 486개 메시지
세션 수: 40개


,#,email,session_id,role,content,created_at
0,1,[EMAIL],4633956e-067a-4803-91c4-463d15b61877,USER,"안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 [...",2025-10-01 06:04:52.205165 +00:00
1,2,[EMAIL],4633956e-067a-4803-91c4-463d15b61877,ASSISTANT,"[NAME]님, 안녕하세요. [DATE] 건강검진 결과를 바탕으로 건강 상태를 자세...",2025-10-01 06:05:30.045710 +00:00
2,3,[EMAIL],4633956e-067a-4803-91c4-463d15b61877,USER,신장(콩팥) 개선 방법을 알려줘,2025-10-01 06:05:37.811683 +00:00
3,4,[EMAIL],4633956e-067a-4803-91c4-463d15b61877,ASSISTANT,"[NAME]님, 단백뇨 관리를 위해 다음과 같은 세 가지 핵심 방법을 꾸준히 실천하...",2025-10-01 06:05:51.403698 +00:00
4,5,[EMAIL],4633956e-067a-4803-91c4-463d15b61877,USER,신장기능 식단 관리 방법을 알려줘,2025-10-01 06:06:11.208222 +00:00


In [10]:
# 플레이스홀더 현황 확인
placeholders = ["[NAME]", "[EMAIL]", "[CONDITION]", "[HEALTH_VALUE]", "[DATE]", "[RESULT]"]
all_content = " ".join(df_masked["content"].astype(str))

print("=== 플레이스홀더 현황 ===")
for ph in placeholders:
    count = all_content.count(ph)
    if count > 0:
        print(f"  {ph}: {count}개")

=== 플레이스홀더 현황 ===
  [NAME]: 203개
  [EMAIL]: 10개
  [CONDITION]: 97개
  [HEALTH_VALUE]: 170개
  [DATE]: 137개
  [RESULT]: 61개


## 6. 전체 재현 데이터 생성

In [11]:
def generate_synthetic_dataframe(df: pd.DataFrame, content_column: str = "content") -> pd.DataFrame:
    """DataFrame의 content 컬럼 전체 재현 데이터 생성"""
    df_synthetic = df.copy()
    
    # email 컬럼도 가상 데이터로 변환
    df_synthetic["email"] = [f"user{i:04d}@example.com" for i in range(len(df_synthetic))]
    
    synthetic_contents = []
    for idx, row in tqdm(df_synthetic.iterrows(), total=len(df_synthetic), desc="재현 데이터 생성"):
        synthetic_content = generate_synthetic_text(row[content_column])
        synthetic_contents.append(synthetic_content)
        
        if (idx + 1) % 10 == 0:
            print(f"[{idx + 1}/{len(df_synthetic)}] 처리 완료")
    
    df_synthetic[content_column] = synthetic_contents
    return df_synthetic

In [12]:
# 테스트: 처음 20개만 처리
df_synthetic = generate_synthetic_dataframe(df_masked.head(20))

# 전체 처리
# df_synthetic = generate_synthetic_dataframe(df_masked)

재현 데이터 생성:  50%|█████     | 10/20 [01:05<00:56,  5.63s/it]

[10/20] 처리 완료


재현 데이터 생성: 100%|██████████| 20/20 [02:02<00:00,  6.14s/it]

[20/20] 처리 완료


## 7. 결과 확인 및 저장

In [13]:
def compare_synthetic(masked_df: pd.DataFrame, synthetic_df: pd.DataFrame, n: int = 3):
    """마스킹 vs 재현 데이터 비교 출력"""
    for i in range(min(n, len(masked_df))):
        print(f"{'='*60}")
        print(f"[{i+1}] 마스킹:")
        print(masked_df.iloc[i]["content"][:500])
        print(f"\n[{i+1}] 재현 데이터:")
        print(synthetic_df.iloc[i]["content"][:500])
    print("=" * 60)

compare_synthetic(df_masked, df_synthetic, n=3)

[1] 마스킹:
안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 [CONDITION], 현재 증상 등을 고려해서 건강 관리 방안을 제안해주세요.

[1] 재현 데이터:
안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 고혈압, 현재 증상 등을 고려해서 건강 관리 방안을 제안해주세요.
[2] 마스킹:
[NAME]님, 안녕하세요. [DATE] 건강검진 결과를 바탕으로 건강 상태를 자세히 살펴보겠습니다.

지난 몇 년간의 검진 결과를 종합적으로 살펴보니, 전반적인 건강 상태는 안정적으로 유지되고 있습니다. 2017년, 2019년, 2021년 모두 ‘[RESULT]’ 또는 ‘[RESULT]’ 판정을 받으셨고, 최근 검진에서도 ‘[RESULT]’ 판정을 받으셨습니다. 주의가 필요한 항목의 수는 약간의 변화는 있었지만, 크게 증가하거나 악화된 부분은 보이지 않습니다. 꾸준히 건강 관리에 신경 쓰신 덕분이라고 생각합니다.

현재 가장 집중적으로 관리해야 할 부분은 단백뇨입니다. 단백뇨는 소변으로 단백질이 배출되는 현상인데, 신장이 제대로 기능을 하고 있는지 확인하는 중요한 지표입니다. 신장은 우리 몸의 노폐물을 걸러내는 역할을 하기 때문에, 단백뇨가 지속되면 신장 기능 저하로 이어질 수 있습니다.

30대 여성의 경우, 단백뇨가 심해지면 혈압 상승, 부종, 피로감 등의 증상이 나타날 수 있

[2] 재현 데이터:
김민지님, 안녕하세요. 2024년 5월 15일 건강검진 결과를 바탕으로 건강 상태를 자세히 살펴보겠습니다.

지난 몇 년간의 검진 결과를 종합적으로 살펴보니, 전반적인 건강 상태는 안정적으로 유지되고 있습니다. 2017년, 2019년, 2021년 모두 ‘정상A’ 또는 ‘정상B’ 판정을 받으셨고, 최근 검진에서도 ‘정상A’ 판정을 받으셨습니다. 주의가 필요한 항목의 수는 약간의 변화는 있었지만, 크게 증가하거나 악화된 부분은 보이지 않습니다. 꾸준히 건강 관리에 신경 쓰신 덕분이라고 생각합니다.

현재 

In [ ]:
output_path = "MESSAGE_INFO_SYNTHETIC.csv"
df_synthetic.to_csv(output_path, index=False)

print(f"재현 데이터 CSV 저장 완료: {output_path}")
print(f"총 {len(df_synthetic)}개 메시지 처리")

재현 데이터 CSV 저장 완료: DB_LIFE_MESSAGE_INFO_SYNTHETIC.csv
총 20개 메시지 처리


## 8. 재현 데이터 품질 검증

In [15]:
def validate_synthetic(df_synthetic: pd.DataFrame) -> dict:
    """재현 데이터 품질 검증"""
    placeholders = [
        "[NAME]", "[EMAIL]", "[PHONE]", "[ADDRESS]", "[SSN]",
        "[CONDITION]", "[HEALTH_VALUE]", "[RESULT]", "[DATE]", "[MEDICATION]"
    ]
    
    all_content = " ".join(df_synthetic["content"].astype(str))
    
    # 남아있는 플레이스홀더 검사
    remaining = {ph: all_content.count(ph) for ph in placeholders if all_content.count(ph) > 0}
    
    # 생성된 이름 패턴 검사 (한글 이름)
    korean_name_pattern = r'[가-힣]{2,4}님'
    names_found = re.findall(korean_name_pattern, all_content)
    unique_names = list(set(names_found))
    
    return {
        "remaining_placeholders": remaining,
        "unique_names_count": len(unique_names),
        "sample_names": unique_names[:10]
    }


result = validate_synthetic(df_synthetic)

print("=== 재현 데이터 검증 결과 ===")
print(f"총 메시지 수: {len(df_synthetic)}")

if result["remaining_placeholders"]:
    print(f"\n[경고] 남은 플레이스홀더:")
    for ph, count in result["remaining_placeholders"].items():
        print(f"  {ph}: {count}개")
else:
    print("\n[OK] 모든 플레이스홀더 변환 완료")

print(f"\n생성된 고유 이름 수: {result['unique_names_count']}개")
print(f"이름 샘플: {result['sample_names']}")

=== 재현 데이터 검증 결과 ===
총 메시지 수: 20

[OK] 모든 플레이스홀더 변환 완료

생성된 고유 이름 수: 2개
이름 샘플: ['김민지님', '김민수님']


## 9. 요약

### 처리 완료
- 마스킹된 CSV -> 재현 데이터 CSV 변환
- 플레이스홀더를 자연스러운 가상 데이터로 대체
- Gemma3:27b 로컬 모델 사용

### 파이프라인
```
원본 CSV (PII 포함)
    ↓ masking_message_example.ipynb
마스킹된 CSV ([NAME], [EMAIL] 등)
    ↓ synthetic_data_generation.ipynb (현재)
재현 데이터 CSV (가상 데이터)
    ↓
외부 공유 / Judge 평가 가능
```

### 다음 단계
- 재현 데이터로 요약 모델 비교
- LLM as a Judge 평가 실행